<a href="https://colab.research.google.com/github/FELIX-NJUGUNA/truthscan/blob/felix/Truthscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#@title Import Modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

import statistics

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from google.colab import drive


# xgboost popular gradient boosting library for increase perfomance and speed
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

from tensorflow.keras import layers
from tensorflow.keras import losses
from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize

from textblob import TextBlob

nltk.download('punkt')  # Download the punkt tokenizer data


# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Ran the import statements.


In [5]:
import tensorflow as tf
print(tf.__version__)

2.18.0


LOADED THE DATASET

In [7]:
!pip install datasets

from datasets import load_dataset

df = load_dataset("ahmadreza13/human-vs-Ai-generated-dataset")

print(df['train'].select(range(5)).to_pandas().head())

                                                data  generated   model
0  def add(a, b):\n    """This function takes two...          1  claude
1  Once upon a time in a faraway kingdom, there l...          1  claude
2  I understand that video games can be very fun ...          1  claude
3  (x - 3)(x - 2) = 0\nTherefore, the solutions a...          1  claude
4  The Importance of Clear and Concise Writing in...          1  claude


In [15]:
#@ creating a copy of the dataset
df_copy = df.copy()
df_copy_train_pd = df_copy['train'].to_pandas() # convert the Hugging Face Dataset to a pandas dataframe
df_copy_train_pd.head()
print(df_copy_train_pd.info()) # use the .info() on the pandas dataframe object


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3614247 entries, 0 to 3614246
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   data       object
 1   generated  int64 
 2   model      object
dtypes: int64(1), object(2)
memory usage: 82.7+ MB
None


In [18]:
#@title check for null values
print(df_copy_train_pd.isnull().sum())

# no null values

data         0
generated    0
model        0
dtype: int64


In [39]:
# Check the data type of the 'generated' column
print("Data type of 'generated':", df_copy_train_pd["generated"].dtype)

# Count the number of AI and human samples
ai_count = df_copy_train_pd[df_copy_train_pd["generated"] == 1].shape[0]
human_count = df_copy_train_pd[df_copy_train_pd["generated"] == 0].shape[0]

print(f"Initial AI count: {ai_count}")
print(f"Initial Human count: {human_count}")

# Balance the dataset
min_count = min(ai_count, human_count)
df_balanced = pd.concat([
    df_copy_train_pd[df_copy_train_pd["generated"] == 1].sample(min_count, random_state=42),
    df_copy_train_pd[df_copy_train_pd["generated"] == 0].sample(min_count, random_state=42)
])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Print the counts of each label after balancing
print(f"Balanced AI count: {df_balanced[df_balanced['generated'] == 1].shape[0]}")
print(f"Balanced Human count: {df_balanced[df_balanced['generated'] == 0].shape[0]}")


#@title Balancing the dataset
# Assuming 'Label' is in the 'train' split
#df_label_human = df_copy_train_pd[df_copy_train_pd['generated'] == 0]  # Use boolean indexing
#df_label_AI = df_copy_train_pd[df_copy_train_pd['generated'] == 1]    # Use boolean indexing

#print(len(df_label_human))
#print(len(df_label_AI))

Data type of 'generated': int64
Initial AI count: 1521889
Initial Human count: 2092358
Balanced AI count: 1521889
Balanced Human count: 1521889


COVERTING TEXT TO NUMERICAL FORMAT

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF vectors
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df_balanced["text"]).toarray()

# Combine with other features
X = np.hstack([X, df_balanced[["text_length", "punctuation_count", "sentiment"]].values])

# Labels
y = df_balanced["label"].map({"ai": 1, "human": 0}).values

KeyError: 'text'

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1)

In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))

In [ ]:
model.save("truthscan_model.h5")

# download the model
from google.colab import files
files.download("truthscan_model.h5")